## Data preparement

In [27]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

"""Read dataframe"""
df1 = pd.read_csv("./ELECTROENCEPHALOGRAM_OF_SENSITIVITY_BODIES/specified_finger.csv",
                  delimiter=";", decimal=".", header=0)
df2 = pd.read_csv("./ELECTROENCEPHALOGRAM_OF_SENSITIVITY_BODIES/thumb_finger.csv",
                  delimiter=";", decimal=".", header=0)

"""Preprocessing data"""
#Addition of target variable
df1["target"] = "specified_finger"
df2["target"] = "thumb_finger"
#Concatenation dataframes

first_10k = df1.iloc[:10000]
second_10k = df2.iloc[-10000:]

frames = [first_10k, second_10k]
df = pd.concat(frames)
#Removing duplicate rows
df.drop_duplicates(subset=None, keep='first', inplace=True)
#Reset index of dataframe
df.reset_index(drop=True, inplace=True)
#Return column names of dataframe
column_names = list(df.columns)
#Checking data balance
target_var = df[column_names[-1]]
balance = Counter(target_var)
#Initialization of encoder
labelencoder = LabelEncoder()
#Encoding target variable
df["target"] = labelencoder.fit_transform(df["target"])


## Splitting to test & train

In [28]:
"""Splitting independent 'x' and dependent 'y'  variables of dataframe 'df'"""
x = df[column_names[0:len(column_names)-1]]
#Standardization or mean removal and variance scaling
x = (x-x.mean())/x.std()
#Converting to numpy array
x = x.values
#Create target variable
y = df[column_names[len(column_names)-1]].values


"""Splitting on training sample and test sample"""
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# KAN model

## Data preparing

In [29]:
import torch

dtype = torch.get_default_dtype()
device = "cpu"

x_train = torch.from_numpy(x_train[:10000]).type(dtype).to(device)
y_train = torch.from_numpy(y_train[:10000]).type(dtype).to(device)

x_test = torch.from_numpy(x_test).type(dtype).to(device)
y_test = torch.from_numpy(y_test).type(dtype).to(device)

custom_dataset = {
    'train_input': x_train,
    'test_input': x_test,
    'train_label': y_train.view(-1, 1),
    'test_label': y_test.view(-1, 1)
}

## KAN model

In [8]:
from kan import KAN

model = KAN(width=[16,2,1], grid=5, k=3)

start_time = time.time()

def train_acc():
    return torch.mean((torch.round(model(custom_dataset['train_input'])[:,0]) == custom_dataset['train_label'][:,0]).type(dtype))

def test_acc():
    return torch.mean((torch.round(model(custom_dataset['test_input'])[:,0]) == custom_dataset['test_label'][:,0]).type(dtype))


print(custom_dataset['train_input'].dtype)
print(custom_dataset['test_input'].dtype)

results = model.fit(custom_dataset, opt="LBFGS", steps=20,
                      metrics=(train_acc, test_acc), beta=7)
end_time = time.time()

results['train_acc'][-1], results['test_acc'][-1]

checkpoint directory created: ./model
saving model version 0.0
torch.float32
torch.float32


| train_loss: 4.33e-01 | test_loss: 4.43e-01 | reg: 3.35e+01 | : 100%|█| 20/20 [00:33<00:00,  1.65s/

saving model version 0.1


(0.7226999998092651, 0.7063999772071838)

In [41]:
logits = model(x_test)
print(logits)
y_pred = torch.round(torch.relu(logits))

tensor([[0.4970],
        [0.4491],
        [0.1912],
        ...,
        [0.3564],
        [0.5995],
        [0.0826]], grad_fn=<AddBackward0>)


In [42]:
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

y_test_numpy = y_test.detach().numpy()
y_pred_numpy = y_pred.detach().numpy().ravel()

print(np.unique(y_test_numpy))
print(np.unique(y_pred_numpy))

#Compute the 'accuracy-score'
acc_score = accuracy_score(y_test_numpy, y_pred_numpy)
#Compute the 'f1-score'
f1 = f1_score(y_test_numpy, y_pred_numpy)
#Compute the 'roc-auc'
roc_auc = roc_auc_score(y_test_numpy, y_pred_numpy)

acc_score, f1, roc_auc

[0. 1.]
[0. 1.]


TypeError: 'float' object is not callable